In [7]:
import numpy as np
import pandas as pd
import string

pd.set_option('display.max_rows', 80)
pd.set_option('display.max_columns', 400)

In [8]:
file_name = "Tab.delimited.Cleaned.dataset.WITH.variable.labels"
df = pd.read_csv(r"ML1/" + file_name + ".preprocessed_hot" + ".csv", sep=',', encoding='utf-8', low_memory=False)

In [33]:
import math
import multiprocessing as mp

def __mutual_info(i, j, col1, col2):
    prob1 = {}
    prob2 = {}
    prob12 = {}
    for val1 in df[col1].unique():
        if pd.isnull(val1):
            prob1["nan"] = (df[col1].isnull().value_counts()[True]/df.shape[0])
            continue
        prob1[val1] = (df[col1].value_counts()[val1]/df.shape[0])
    #         print(val1, " ", prob1[val1])
    #       print()
    for val2 in df[col2].unique():
        if pd.isnull(val2):
            prob2["nan"] = (df[col2].isnull().value_counts()[True]/df.shape[0])
            continue
        prob2[val2] = (df[col2].value_counts()[val2]/df.shape[0])
    #         print(val2, " ", prob2[val2])
    for val1 in df[col1].unique():
        #         if pd.isnull(val1):
        #           continue
        for val2 in df[col2].unique():
            if pd.isnull(val1) and not pd.isnull(val2):
                prob12[("nan", val2)] = df[df[col1].isnull() & (df[col2] == val2)].shape[0]/df.shape[0]
            elif not pd.isnull(val1) and pd.isnull(val2):
                prob12[(val1, "nan")] = df[(df[col1] == val1) & df[col2].isnull()].shape[0]/df.shape[0]
            elif pd.isnull(val1) and pd.isnull(val2):
                prob12[("nan", "nan")] = df[df[col1].isnull() & df[col2].isnull()].shape[0]/df.shape[0]
            else:
                prob12[(val1, val2)] = df[(df[col1] == val1) & (df[col2] == val2)].shape[0]/df.shape[0]
    #           print(val1,  " ", val2, " ", prob12[(val1, val2)])
    #       print()
    mutual_in = 0
    for k in range(df.shape[0]):
        if pd.isnull(df.iloc[k, i]) and not pd.isnull(df.iloc[k, j]):
            ratio = prob12[("nan", df.iloc[k, j])]/(prob1["nan"]*prob2[df.iloc[k, j]])
        elif not pd.isnull(df.iloc[k, i]) and pd.isnull(df.iloc[k, j]):
            ratio = prob12[(df.iloc[k, i], "nan")]/(prob1[df.iloc[k, i]]*prob2["nan"])
        elif pd.isnull(df.iloc[k, i]) and pd.isnull(df.iloc[k, j]):
            ratio = prob12[("nan", "nan")]/(prob1["nan"]*prob2["nan"])
        else:
            ratio = prob12[(df.iloc[k, i], df.iloc[k, j])]/(prob1[df.iloc[k, i]]*prob2[df.iloc[k, j]])
        if ratio == 0:
            continue
        mutual_in = mutual_in + math.log(ratio)
    return mutual_in

col_iterable = []
def mutual_info(df):
    mutual_info_matrix = np.zeros(shape = (df.shape[1], df.shape[1]))
    pool = mp.Pool(processes=mp.cpu_count() - 1)
    for i, col1 in enumerate(list(df)):
        for j, col2 in enumerate(list(df)):
            if j < i:
                continue
            col_iterable.append((i, j, col1, col2))
    for i in range(int(len(col_iterable)/(mp.cpu_count()))):
        iterable = col_iterable[i*(mp.cpu_count()): (i+1)*(mp.cpu_count())]
        ans = pool.starmap(__mutual_info, iterable)
        for element in list(zip(iterable, ans)):
            mutual_info_matrix[element[0][0]][element[0][1]] = element[1]
            print("Mutual Info for ", element[0][0], " ", element[0][1], " ", element[0][2], " ", element[0][3], \
                 " is: ", element[1])        
    return mutual_info_matrix

In [ ]:
arr = mutual_info(df)

Mutual Info for  0   0   numparticipants_actual   numparticipants_actual  is:  1428.4989410544579
Mutual Info for  0   1   numparticipants_actual   numparticipants  is:  131.9078692122571
Mutual Info for  0   2   numparticipants_actual   expgender  is:  347.2559011416775
Mutual Info for  0   3   numparticipants_actual   exprace  is:  583.293361504278
Mutual Info for  0   4   numparticipants_actual   exprunafter  is:  234.31066684034363
Mutual Info for  0   5   numparticipants_actual   exprunafter2  is:  26.91796496592068
Mutual Info for  0   6   numparticipants_actual   compensation  is:  220.32205199783866
Mutual Info for  0   7   numparticipants_actual   recruitment  is:  188.8659315303356
Mutual Info for  0   8   numparticipants_actual   separatedornot  is:  323.35558267241254
Mutual Info for  0   9   numparticipants_actual   age  is:  263.7095109944971
Mutual Info for  0   10   numparticipants_actual   sample  is:  1104.114132691816
Mutual Info for  0   11   numparticipants_actual 

Mutual Info for  0   96   numparticipants_actual   iatexplicitart5  is:  39.811466302620644
Mutual Info for  0   97   numparticipants_actual   iatexplicitart6  is:  33.86509039689697
Mutual Info for  0   98   numparticipants_actual   iatexplicitmath1  is:  35.961269172563405
Mutual Info for  0   99   numparticipants_actual   iatexplicitmath2  is:  34.8923830217144
Mutual Info for  0   100   numparticipants_actual   iatexplicitmath3  is:  40.52247390877375
Mutual Info for  0   101   numparticipants_actual   iatexplicitmath4  is:  27.201631590998858
Mutual Info for  0   102   numparticipants_actual   iatexplicitmath5  is:  36.317707670908256
Mutual Info for  0   103   numparticipants_actual   iatexplicitmath6  is:  30.313296189882372
Mutual Info for  0   104   numparticipants_actual   imaginedexplicit1  is:  57.68178121274295
Mutual Info for  0   105   numparticipants_actual   imaginedexplicit2  is:  44.49844800260771
Mutual Info for  0   106   numparticipants_actual   imaginedexplicit3 

Mutual Info for  1   2   numparticipants   expgender  is:  3919.4237746341314
Mutual Info for  1   3   numparticipants   exprace  is:  4872.765485689488
Mutual Info for  1   4   numparticipants   exprunafter  is:  3945.2446867867184
Mutual Info for  1   5   numparticipants   exprunafter2  is:  1857.6686441907666
Mutual Info for  1   6   numparticipants   compensation  is:  4545.115997628979
Mutual Info for  1   7   numparticipants   recruitment  is:  3747.050508930661
Mutual Info for  1   8   numparticipants   separatedornot  is:  4200.090367693932
Mutual Info for  1   9   numparticipants   age  is:  1488.977590052079
Mutual Info for  1   10   numparticipants   sample  is:  7710.575392431773
Mutual Info for  1   11   numparticipants   sunkgroup  is:  15.89465927576267
Mutual Info for  1   12   numparticipants   sunkDV  is:  162.74236725085555
Mutual Info for  1   13   numparticipants   gainlossgroup  is:  17.21369124348204
Mutual Info for  1   14   numparticipants   gainlossDV  is:  26

Mutual Info for  1   106   numparticipants   imaginedexplicit3  is:  133.109092428464
Mutual Info for  1   107   numparticipants   imaginedexplicit4  is:  151.45069122195238
Mutual Info for  1   108   numparticipants   imptaskto  is:  10.224591873903803
Mutual Info for  1   109   numparticipants   major  is:  1457.9253157385988
Mutual Info for  1   110   numparticipants   mathwarm  is:  1078.298414931246
Mutual Info for  1   111   numparticipants   moneyagea  is:  838.9632990807414
Mutual Info for  1   112   numparticipants   moneygendera  is:  36.44688332789684
Mutual Info for  1   113   numparticipants   moneygenderb  is:  43.91794699217909
Mutual Info for  1   114   numparticipants   nativelang  is:  1113.4261925590597
Mutual Info for  1   115   numparticipants   noflagtimeestimate1  is:  35.844345913920485
Mutual Info for  1   116   numparticipants   noflagtimeestimate2  is:  49.307199444396616
Mutual Info for  1   117   numparticipants   noflagtimeestimate3  is:  52.93827924104637

Mutual Info for  2   21   expgender   anchoring3  is:  1077.6170399612915
Mutual Info for  2   22   expgender   anchoring4  is:  442.389840137046
Mutual Info for  2   23   expgender   Ranch1  is:  687.3022497532961
Mutual Info for  2   24   expgender   Ranch2  is:  555.505855011245
Mutual Info for  2   25   expgender   Ranch3  is:  1077.6170399612915
Mutual Info for  2   26   expgender   Ranch4  is:  442.389840137046
Mutual Info for  2   27   expgender   gambfalgroup  is:  1.9507779884051277
Mutual Info for  2   28   expgender   gambfalDV  is:  148.4789132066937
Mutual Info for  2   29   expgender   scalesgroup  is:  0.15548328777200976
Mutual Info for  2   30   expgender   scalesreca  is:  34.595928351749805
Mutual Info for  2   31   expgender   scalesrecb  is:  13.481326822623338
Mutual Info for  2   32   expgender   scales  is:  41.221673399773636
Mutual Info for  2   33   expgender   reciprocitygroup  is:  0.46352484577719055
Mutual Info for  2   34   expgender   reciprocityother  

...

Mutual Info for  18   37   anch4group   allowedforbidden  is:  0.07897995530965193
Mutual Info for  18   38   anch4group   quotearec  is:  1.5814533044886327
Mutual Info for  18   39   anch4group   quotebrec  is:  7.788730298217784
Mutual Info for  18   40   anch4group   quoteGroup  is:  2.386289692629926e-05
Mutual Info for  18   41   anch4group   quote  is:  5.958828238806153
Mutual Info for  18   42   anch4group   totalflagestimations  is:  2.121466903384694
Mutual Info for  18   43   anch4group   totalnoflagtimeestimations  is:  5.41380473755751
Mutual Info for  18   44   anch4group   flagfilter  is:  11.58622068799012
Mutual Info for  18   45   anch4group   flagdv  is:  38.56561927146662
Mutual Info for  18   46   anch4group   flagGroup  is:  0.5406592459609302
Mutual Info for  18   47   anch4group   Sysjust  is:  46.83221513620532
Mutual Info for  18   48   anch4group   MoneyGroup  is:  0.010173376761089968
Mutual Info for  18   49   anch4group   moneyfilter  is:  0.1163438396987

Mutual Info for  18   149   anch4group   priorexposure12  is:  5.45497753267299
Mutual Info for  18   150   anch4group   priorexposure13  is:  3.907081793509151
Mutual Info for  18   151   anch4group   priorexposure2  is:  6.100588714323423
Mutual Info for  18   152   anch4group   priorexposure3  is:  4.281537862795366
Mutual Info for  18   153   anch4group   priorexposure4  is:  4.358796332103702
Mutual Info for  18   154   anch4group   priorexposure5  is:  3.635007539429529
Mutual Info for  18   155   anch4group   priorexposure6  is:  4.180552379587929
Mutual Info for  18   156   anch4group   priorexposure7  is:  6.8683609106322905
Mutual Info for  18   157   anch4group   priorexposure8  is:  6.184965902929743
Mutual Info for  18   158   anch4group   priorexposure9  is:  9.011661697000635
Mutual Info for  18   159   anch4group   mturk.Submitted.PaymentReq  is:  0.5718891917445197
Mutual Info for  18   160   anch4group   mturk.total.mini.exps  is:  1.9424934661399882
Mutual Info for  

Mutual Info for  19   89   anchoring1   flagtimeestimate4  is:  538.8382243694812
Mutual Info for  19   90   anchoring1   gamblerfallacya  is:  2043.3114548007647
Mutual Info for  19   91   anchoring1   gamblerfallacyb  is:  1687.675594703383
Mutual Info for  19   92   anchoring1   iatexplicitart1  is:  774.0109796039226
Mutual Info for  19   93   anchoring1   iatexplicitart2  is:  784.1953880609873
Mutual Info for  19   94   anchoring1   iatexplicitart3  is:  705.1368515848723
Mutual Info for  19   95   anchoring1   iatexplicitart4  is:  773.6477415558776
Mutual Info for  19   96   anchoring1   iatexplicitart5  is:  836.4687079729774
Mutual Info for  19   97   anchoring1   iatexplicitart6  is:  807.2347316190802
Mutual Info for  19   98   anchoring1   iatexplicitmath1  is:  922.4596782104841
Mutual Info for  19   99   anchoring1   iatexplicitmath2  is:  967.8931613221963
Mutual Info for  19   100   anchoring1   iatexplicitmath3  is:  959.8611875886401
Mutual Info for  19   101   ancho

Mutual Info for  20   30   anchoring2   scalesreca  is:  383.5070290709142
Mutual Info for  20   31   anchoring2   scalesrecb  is:  487.5420464921176
Mutual Info for  20   32   anchoring2   scales  is:  380.9643165548896
Mutual Info for  20   33   anchoring2   reciprocitygroup  is:  302.0614708668134
Mutual Info for  20   34   anchoring2   reciprocityother  is:  359.6866271909671
Mutual Info for  20   35   anchoring2   reciprocityus  is:  393.10187031630727
Mutual Info for  20   36   anchoring2   allowedforbiddenGroup  is:  306.65707189055865
Mutual Info for  20   37   anchoring2   allowedforbidden  is:  335.81781909458005
Mutual Info for  20   38   anchoring2   quotearec  is:  998.8989971862945
Mutual Info for  20   39   anchoring2   quotebrec  is:  1079.2306259245295
Mutual Info for  20   40   anchoring2   quoteGroup  is:  313.1933065058626
Mutual Info for  20   41   anchoring2   quote  is:  1179.43876131007
Mutual Info for  20   42   anchoring2   totalflagestimations  is:  813.43272

Mutual Info for  20   142   anchoring2   study_url  is:  1769.6872130847182
Mutual Info for  20   143   anchoring2   study_name  is:  1769.6872130847182
Mutual Info for  20   144   anchoring2   us_or_international  is:  332.05352597493174
Mutual Info for  20   145   anchoring2   lab_or_online  is:  356.8326996797112
Mutual Info for  20   146   anchoring2   priorexposure1  is:  463.4367009696861
Mutual Info for  20   147   anchoring2   priorexposure10  is:  468.57997976906233
Mutual Info for  20   148   anchoring2   priorexposure11  is:  488.673870251623
Mutual Info for  20   149   anchoring2   priorexposure12  is:  482.4075265161266
Mutual Info for  20   150   anchoring2   priorexposure13  is:  501.96758257332084
Mutual Info for  20   151   anchoring2   priorexposure2  is:  490.3566095189082
Mutual Info for  20   152   anchoring2   priorexposure3  is:  473.3309821070768
Mutual Info for  20   153   anchoring2   priorexposure4  is:  455.2347004952755
Mutual Info for  20   154   anchoring

In [134]:
import pickle as pk
with open(r"mutual_info_matrix.pkl", "wb") as f:
    pk.dump(arr, f)

In [135]:
import numpy as np
np.set_printoptions(suppress=True)

parent_array = []
#def __init__(self):
    #self.parent_array = []

def quick_union(p, q, parent_array):
    pid = parent_array[p]
    qid = parent_array[q]
    
    for k in range(len(parent_array)):
        if parent_array[k] == pid:
            parent_array[k] = qid

with open(r"mutual_info_matrix.pkl", "rb") as f:
    arr_temp = pk.load(f)
    
print(arr_temp)    

arr2 = []
for i in range(len(arr_temp)):
    arr3 = []
    j = 0
    for j in range(len(arr_temp[i])):
        if arr_temp[i][j] > 300:
            arr3.append(1)
        else:
            arr3.append(0)
    arr2.append(arr3)        
            
#print(arr2[31][30])    

    
#arr_test = [[1, 0, 0, 0, 0, 0, 1, 1],
            #[0, 1, 0, 0, 1, 1, 0, 0],
            #[0, 0, 0, 1, 0, 0, 0, 0],
            #[0, 0, 1, 0, 0, 0, 0, 0],
            #[0, 1, 0, 0, 1, 1, 0, 1],
            #[0, 1, 0, 0, 1, 1, 1, 0],
            #[1, 0, 0, 0, 0, 1, 1, 1],
            #[1, 0, 0, 0, 1, 0, 1, 1]]

for i in range(len(arr2)):
    parent_array.append(i)
    
#for i in range(len(parent_array)):
    #for j in range(i, len(parent_array), 1):
        #if arr_temp[i][j] > 500:
            #quick_union(i, j, parent_array)
            
for i in range(len(arr2)):
    for j in range(i, len(arr2), 1):
        if arr2[i][j] == 1:
            quick_union(i, j, parent_array)
            
temppp = ((np.array(parent_array)))
print(temppp)

[[ 6311.21809803   185.77867234     6.83605324 ...     8.78995543
      4.88637242     7.25049581]
 [    0.          6551.26660341    37.03817562 ...    56.48153353
     57.51749793    59.49968224]
 [    0.             0.          7514.48273015 ...    16.52431621
     14.45581229    14.35909373]
 ...
 [    0.             0.             0.         ... 15200.87025594
    646.48438501   662.38921989]
 [    0.             0.             0.         ...     0.
  15206.14420553   652.084128  ]
 [    0.             0.             0.         ...     0.
      0.         15205.9610683 ]]
[144 177 132 132 132 132 133 177 136 136 177 141 141 141 141 143 143 144
 146 146 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177
 177 177 177 177 177 177 177 177 177 177 177 177 177 177 177 178 177 177
 177 177 204 204 204 204 204 204 204 204 204 204 204 204 204 204 204 204
 204 204 204 204 204 204]
[129, 130, 131, 132]


In [136]:
for i in range(len(_132_set)):
    print(_132_set[i], " ", df.iloc[:, i].isnull().any())

129   False
130   False
131   False
132   False


In [139]:
df.iloc[:, _177_set].isnull().any()

nativelang                     True
politicalid                    True
race                           True
sysjust1                       True
sysjust2                       True
sysjust3                       True
sysjust4                       True
sysjust5                       True
sysjust6                       True
sysjust7                       True
sysjust8                       True
text                           True
study_url                     False
study_name                    False
us_or_international           False
lab_or_online                 False
priorexposure1                 True
priorexposure10                True
priorexposure11                True
priorexposure12                True
priorexposure13                True
priorexposure2                 True
priorexposure3                 True
priorexposure4                 True
priorexposure5                 True
priorexposure6                 True
priorexposure7                 True
priorexposure8              

In [121]:
print(df.iloc[:, 127:].isnull().any())

moneygenderb                  False
nativelang                    False
noflagtimeestimate1            True
noflagtimeestimate2            True
noflagtimeestimate3            True
noflagtimeestimate4            True
omdimc3                       False
politicalid                   False
quotea                        False
quoteb                        False
race                          False
reciprocityothera             False
reciprocityotherb             False
reciprocityusa                False
reciprocityusb                False
scalesa                        True
scalesb                        True
sex                           False
sunkcosta                     False
sunkcostb                     False
sysjust1                       True
sysjust2                       True
sysjust3                       True
sysjust4                       True
sysjust5                       True
sysjust6                       True
sysjust7                       True
sysjust8                    

In [ ]:
def calculate_info_gain_g(df, col, y, consider_nan = "True"):
  if consider_nan:
    y_value_counts = df.iloc[:, y].value_counts().to_dict()
    y_value_counts["nan"] = df.shape[0] - df.iloc[:, y].value_counts().sum()
    col_value_counts = df.iloc[:, col].value_counts().to_dict()
    col_value_counts["nan"] = df.shape[0] - df.iloc[:, col].value_counts().sum()
    h_y = 0
    h_x_y = 0
    count_dict = {}
    
    for val1 in df.iloc[:, col].unique():
      for val2 in df.iloc[:, y].unique():
        if pd.isnull(val1) and pd.isnull(val2):
          count_dict[("nan", "nan")] = np.count_nonzero((df.iloc[:, col].isnull()) & (df.iloc[:, y].isnull()))
        elif pd.isnull(val1) and not pd.isnull(val2):
          count_dict[("nan", val2)] = np.count_nonzero((df.iloc[:, col].isnull()) & (df.iloc[:, y] == val2))
        elif not pd.isnull(val1) and pd.isnull(val2):
          count_dict[(val1, "nan")] = np.count_nonzero((df.iloc[:, col] == val1) & (df.iloc[:, y].isnull()))
        elif not pd.isnull(val1) and not pd.isnull(val2):
          count_dict[(val1, val2)] = np.count_nonzero((df.iloc[:, col] == val1) & (df.iloc[:, y] == val2))
    for val in y_value_counts.keys():
      if y_value_counts[val] != 0:
        h_y = h_y - y_value_counts[val]/df.shape[0] * math.log(y_value_counts[val]/df.shape[0])
    
    for val1 in df.iloc[:, col].unique():
      for val2 in df.iloc[:, y].unique():
        if pd.isnull(val1) and pd.isnull(val2) and count_dict[("nan", "nan")] != 0:
          h_x_y = h_x_y - col_value_counts["nan"]/df.shape[0]*count_dict[("nan", "nan")]/col_value_counts["nan"]*\
          math.log(count_dict[("nan", "nan")]/col_value_counts["nan"])
        elif pd.isnull(val1) and not pd.isnull(val2) and count_dict[("nan", val2)] != 0:
          h_x_y = h_x_y - col_value_counts["nan"]/df.shape[0]*count_dict[("nan", val2)]/col_value_counts["nan"]*\
          math.log(count_dict[("nan", val2)]/col_value_counts["nan"])
        elif not pd.isnull(val1) and pd.isnull(val2) and count_dict[(val1, "nan")] != 0:
          h_x_y = h_x_y - col_value_counts[val1]/df.shape[0]*count_dict[(val1, "nan")]/col_value_counts[val1]*\
          math.log(count_dict[(val1, "nan")]/col_value_counts[val1])
        elif not pd.isnull(val1) and not pd.isnull(val2) and count_dict[(val1, val2)] != 0:
          h_x_y = h_x_y - col_value_counts[val1]/df.shape[0]*count_dict[(val1, val2)]/col_value_counts[val1]*\
          math.log(count_dict[(val1, val2)]/col_value_counts[val1])  
  else:
    y_value_counts = df.iloc[:, y].value_counts()
#     y_value_counts["nan"] = df.shape[0] - df.iloc[:, y].value_counts().sum()
    col_value_counts = df.iloc[:, col].value_counts()
#     col_value_counts["nan"] = df.shape[0] - df.iloc[:, col].value_counts().sum()
    h_y = 0
    h_x_y = 0
    count_dict = {}
    
    for val1 in df.iloc[:, col].unique():
      for val2 in df.iloc[:, y].unique():
        if pd.isnull(val1) and pd.isnull(val2):
#           count_dict[("nan", "nan")] = np.count_nonzero((df.iloc[:, col].isnull()) & (df.iloc[:, y].isnull()))
          pass
        elif pd.isnull(val1) and not pd.isnull(val2):
#           count_dict[("nan", val2)] = np.count_nonzero((df.iloc[:, col].isnull()) & (df.iloc[:, y] == val2))
          pass
        elif not pd.isnull(val1) and pd.isnull(val2):
#           count_dict[(val1, "nan")] = np.count_nonzero((df.iloc[:, col] == val1) & (df.iloc[:, y].isnull()))
          pass
        elif not pd.isnull(val1) and not pd.isnull(val2):
          count_dict[(val1, val2)] = np.count_nonzero((df.iloc[:, col] == val1) & (df.iloc[:, y] == val2))
      
    for val in y_value_counts.keys():
      if y_value_counts[val] != 0:
        h_y = h_y - y_value_counts[val]/df.iloc[:, y].value_counts().sum()\
        * math.log(y_value_counts[val]/df.iloc[:, y].value_counts().sum())
    
    for val1 in df.iloc[:, col].unique():
      for val2 in df.iloc[:, y].unique():
        if pd.isnull(val1) and pd.isnull(val2):
#           h_x_y = h_x_y - col_value_counts["nan"]/df.shape[0]*count_dict[("nan", "nan")]/col_value_counts["nan"]*\
#           math.log(count_dict[("nan", "nan")]/col_value_counts["nan"])
          pass
        elif pd.isnull(val1) and not pd.isnull(val2):
#           h_x_y = h_x_y - col_value_counts["nan"]/df.shape[0]*count_dict[("nan", val2)]/col_value_counts["nan"]*\
#           math.log(count_dict[("nan", val2)]/col_value_counts["nan"])
          pass
        elif not pd.isnull(val1) and pd.isnull(val2):
#           h_x_y = h_x_y - col_value_counts[val1]/df.shape[0]*count_dict[(val1, "nan")]/col_value_counts[val1]*\
#           math.log(count_dict[(val1, "nan")]/col_value_counts[val1])
          pass
        elif not pd.isnull(val1) and not pd.isnull(val2) and count_dict[(val1, val2)] != 0:
          h_x_y = h_x_y - col_value_counts[val1]/df.iloc[:, col].value_counts().sum()\
      *count_dict[(val1, val2)]/col_value_counts[val1]*\
          math.log(count_dict[(val1, val2)]/col_value_counts[val1])
          
  return h_y - h_x_y

In [ ]:
calculate_info_gain_g(df_temp, 0, 2, False)